In [67]:
import numpy as np
from tqdm import tqdm
import time

In [68]:
#function to check if the puzzle is solved
def compare(lst1,lst2):
    #loop that checks if every element of the current state matches the goal state
    for i in range(len(lst1)):
        if(lst1[i]!=lst2[i]):
            return False
    return True

In [69]:
#function to swap the blank space(0) with the number on top of it
def move_up(cur):
    pos=list(cur).index(0)
    #check whether the swap is possible (i.e. the blank can move up)
    if(int(pos/3)!=0):
        cur[pos],cur[pos-3]=cur[pos-3],cur[pos]
    return np.reshape(cur,(3,3))

In [70]:
#function to swap the blank space(0) with the number on the left of it
def move_left(cur):
    pos=list(cur).index(0)
    #check whether the swap is possible (i.e. the blank can move left)
    if(int(pos%3)!=0):
        cur[pos],cur[pos-1]=cur[pos-1],cur[pos]
    return np.reshape(cur,(3,3))

In [71]:
#function to swap the blank space(0) with the numbee on the right of it
def move_right(cur):
    pos=list(cur).index(0)
    #check whether the swap is possible (i.e. the blank can move right)
    if(int(pos%3)!=2):
        cur[pos],cur[pos+1]=cur[pos+1],cur[pos]
    return np.reshape(cur,(3,3))

In [72]:
#function to swap the blank space(0) with the number below it
def move_down(cur):
    pos=list(cur).index(0)
    #check whether the swap is possible (i.e. the blank can move down)
    if(int(pos/3)!=2):
        cur[pos],cur[pos+3]=cur[pos+3],cur[pos]
    return np.reshape(cur,(3,3))

In [73]:
#function to calculate manhattan distance for the current state
def manhattan_dist(state,goal):
    d=[]
    #loop over every element in the state and compare with the goal state
    for i in range(0,len(state)):
        for j in range(0,len(state[i])):
            #check if the number is in the correct position
            if state[i][j]==goal[i][j]:
                continue
            #don't count the manhattan distance for the blank
            if state[i][j]==0:
                continue
            else:
                #retrieve the position from the goal state
                ig,jg=lookup(state[i][j],goal)
                dist=abs(i-ig)+abs(j-jg)
                d.append(dist)
    return sum(d)

In [74]:
#function called from manhattan_dist to retrieve the i,j for a particular number from the goal state
def lookup(n,goal):
    for i in range(0,len(goal)):
        for j in range(0,len(goal[i])):
            if n==goal[i][j]:
                return i,j
    return None,None

In [75]:
#function to count the number of misplaced tiles
def misplaced_tile(state,goal):
    ctr=0
    for i in range(0,len(state)):
        for j in range(0,len(state[i])):
            if state[i][j]==0:
                continue
            if state[i][j]!=goal[i][j]:
                ctr+=1
    return ctr

In [102]:
#uniform cost search(no heuristic involved)
def uniform_cost(start,goal):
    depth=[]
    nodes=[]
    sucmsg="Success"
    failmsg="Failure"
    nodes_expanded=0
    visited_nodes=[]
    starttime=time.time()
    #check if the start state is already in the goal state
    if(compare(start.flatten(),goal.flatten())):
        print(start)
        print("Solution found at depth 0")
        print("Nodes expanded: ",nodes_expanded)
        end=time.time()-starttime
        print("Solution found in: {:.4f}".format(end)," seconds")
        return sucmsg
    #if start state is not the goal state add it to the list of states to be expanded
    nodes.append(start)
    #keeping track of the current depth using a list
    depth.append(0)
    #infinite loop to keep the search going until the solution is found
    while True:
        #terminal condition for the loop when there are no states left to traverse (i.e. the solution does not exist)
        if(len(nodes)==0):
            return failmsg
        #popping the states in order for breadth first search
        node=nodes.pop(0)
        curdepth=depth.pop(0)
        #keeping track of the visited states and preventing the expansion of repeated states
        if tuple(node.flatten()) in visited_nodes:
            continue
        nodes_expanded+=1
        visited_nodes.append(tuple(node.flatten()))
        #printing the traversal details for the states (uncomment if you want to see the trace as it crashes for problems at high depth)
#         print(node)
#         print("Depth:",curdepth)
#         print("\n")
        #if the current state matches the goal state, then exit
        if(compare(node.flatten(),goal.flatten())):
            print("Success")
            print("Nodes expanded: ",nodes_expanded)
            print("Depth:",curdepth)
            end=time.time()-starttime
            print("Solution found in: {:.4f}".format(end)," seconds")
            return sucmsg
        #adding the results of moving the blank to the list of states to be expanded
        next=move_up(node.flatten())
        if not compare(next.flatten(),node.flatten()):
            nodes.append(next)
            depth.append(curdepth+1)
        next=move_down(node.flatten())
        if not compare(next.flatten(),node.flatten()):
            nodes.append(next)
            depth.append(curdepth+1)
        next=move_left(node.flatten())
        if not compare(next.flatten(),node.flatten()):
            nodes.append(next)
            depth.append(curdepth+1)
        next=move_right(node.flatten())
        if not compare(next.flatten(),node.flatten()):
            nodes.append(next)
            depth.append(curdepth+1)

In [109]:
#A* search using the misplaced tile heuristic
def a_star_mtile(state, goal):
    depth=[]
    d=0
    nodes=[]
    sucmsg="Success"
    failmsg="Failure"
    nodes_expanded=0
    visited_nodes=[]
    starttime=time.time()
    #check if the start state is already in the goal state
    if(compare(state.flatten(),goal.flatten())):
        print(state)
        print("Solution found at depth 0")
        print("Nodes expanded: ",nodes_expanded)
        end=time.time()-starttime
        print("Solution found in: {:.4f}".format(end)," seconds")
        return sucmsg
    #if start state is not the goal state add it to the list of states to be expanded
    nodes.append(state)
    #keeping track of the current depth using a list
    depth.append(d)
    #infinite loop to keep the search going until the solution is found
    while True:
        #terminal condition for the loop when there are no states left to traverse (i.e. the solution does not exist)
        if(len(nodes)==0):
            return failmsg
        #calculating the misplaced tile heuristic for every state in the list of states
        h=[misplaced_tile(s,goal) for s in nodes]
        #finding the state with the least value for the heuristic
        x=h.index(min(h))
        #popping the corresponding depth for the current state
        curdepth=depth.pop(x)
        #popping the state with the minimum value for the misplaced tile heuristic for A* search
        node=nodes.pop(x)
        #keeping track of the visited states and preventing the expansion of repeated states
        if tuple(node.flatten()) in visited_nodes:
            continue
        nodes_expanded+=1
        #printing the traversal details for the states
        print(node)
        print("Misplaced Tile Heuristic: ",h[x])
        print("Depth:",curdepth)
        print("\n")
        visited_nodes.append(tuple(node.flatten()))
        #if the current state matches the goal state, then exit
        if(h[x]==0):
            print("Success")
            print("Nodes expanded: ",nodes_expanded)
            print("Depth: ",curdepth)
            end=time.time()-starttime
            print("Solution found in: {:.4f}".format(end)," seconds")
            return sucmsg
        #adding the results of moving the blank to the list of states to be expanded only if the heuristic for the next state is less than or equal to the heuristic for the current state
        next=move_up(node.flatten())
        newh=misplaced_tile(next,goal)
        if newh<=h[x]:
            nodes.append(next)
            depth.append(curdepth+1)
        next=move_down(node.flatten())
        newh=misplaced_tile(next,goal)
        if newh<=h[x]:
            nodes.append(next)
            depth.append(curdepth+1)
        next=move_left(node.flatten())
        newh=misplaced_tile(next,goal)
        if newh<=h[x]:
            nodes.append(next)
            depth.append(curdepth+1)
        next=move_right(node.flatten())
        newh=misplaced_tile(next,goal)
        if newh<=h[x]:
            nodes.append(next)
            depth.append(curdepth+1)

In [110]:
#A* search using the manhattan distance heuristic
def a_star_man(state, goal):
    depth=[]
    d=0
    nodes=[]
    sucmsg="Success"
    failmsg="Failure"
    nodes_expanded=0
    visited_nodes=[]
    starttime=time.time()
    #check if the start state is already in the goal state
    if(compare(state.flatten(),goal.flatten())):
        print(state)
        print("Solution found at depth 0")
        print("Nodes expanded: ",nodes_expanded)
        end=time.time()-starttime
        print("Solution found in: {:.4f}".format(end)," seconds")
        return sucmsg
    #if start state is not the goal state add it to the list of states to be expanded
    nodes.append(state)
    #keeping track of the current depth using a list
    depth.append(d)
    #infinite loop to keep the search going until the solution is found
    while True:
        #terminal condition for the loop when there are no states left to traverse (i.e. the solution does not exist)
        if(len(nodes)==0):
            return failmsg
        #calculating the Manhattan distance heuristic for every state in the list of states
        h=[manhattan_dist(s,goal) for s in nodes]
        #finding the state with the least value for the heuristic
        x=h.index(min(h))
        #popping the corresponding depth for the current state
        curdepth=depth.pop(x)
        #popping the state with the minimum value for the misplaced tile heuristic for A* search
        node=nodes.pop(x)
        #keeping track of the visited states and preventing the expansion of repeated states
        if tuple(node.flatten()) in visited_nodes:
            continue
        nodes_expanded+=1
        #printing the traversal details for the states
        print(node)
        print(h[x])
        print("Depth:",curdepth)
        print("\n")
        visited_nodes.append(tuple(node.flatten()))
        #if the current state matches the goal state, then exit
        if(h[x]==0):
            print("Success")
            print("Nodes expanded: ",nodes_expanded)
            print("Depth: ",curdepth)
            end=time.time()-starttime
            print("Solution found in: {:.4f}".format(end)," seconds")
            return sucmsg
        #adding the results of moving the blank to the list of states to be expanded only if the heuristic for the next state is less than or equal to the heuristic for the current state
        next=move_up(node.flatten())
        newh=misplaced_tile(next,goal)
        if newh<=h[x]:
            nodes.append(next)
            depth.append(curdepth+1)
        next=move_down(node.flatten())
        newh=misplaced_tile(next,goal)
        if newh<=h[x]:
            nodes.append(next)
            depth.append(curdepth+1)
        next=move_left(node.flatten())
        newh=misplaced_tile(next,goal)
        if newh<=h[x]:
            nodes.append(next)
            depth.append(curdepth+1)
        next=move_right(node.flatten())
        newh=misplaced_tile(next,goal)
        if newh<=h[x]:
            nodes.append(next)
            depth.append(curdepth+1)

In [99]:
#driver function
def main():
    #few simple testing problems
    trivial=np.reshape(np.array([1,2,3,4,5,6,7,8,0]),(3,3))
    veasy=np.reshape(np.array([1,2,3,4,5,6,7,0,8]),(3,3))
    easy=np.reshape(np.array([1,2,0,4,5,3,7,8,6]),(3,3))
    doable=np.reshape(np.array([0,1,2,4,5,3,7,8,6]),(3,3))
    ohboy=np.reshape(np.array([8,7,1,6,0,2,5,4,3]),(3,3))
    print("Enter the difficulty of puzzle:","\n","1. Trivial","\n","2. Very Easy","\n","3. Easy","\n","4. Doable","\n","5. Hard","\n","6. Custom")
    #input choice of puzzle
    ch=int(input("Enter you choice: "))
    if ch==1:
        start=trivial
    elif ch==2:
        start=veasy
    elif ch==3:
        start=easy
    elif ch==4:
        start=doable
    elif ch==5:
        start=ohboy
    else:
        #reading inputs from the user for a custom puzzle
        print("Enter the puzzle using 0 to indicate the blank. Please enter only valid puzzles with the numbers separated by spaces.")
        lst=input("Enter first row: ")
        lst1=input("Enter second row: ")
        lst2=input("Enter third row: ")    
        tmp=[]
        for item in lst.split():
            tmp.append(int(item))
        for item in lst1.split():
            tmp.append(int(item))
        for item in lst2.split():
            tmp.append(int(item))
        start=np.reshape(np.array(tmp),(3,3))
    #Printing the inital state
    print("Start state:")
    print(start)
    #Setting the goal state and printing it
    goal=np.reshape(np.array([1,2,3,4,5,6,7,8,0]),(3,3))
    print("Goal state:")
    print(goal)
    #taking the choice of algorithm to be run from the user
    print("Enter the choice of algorithm you wish to run-","\n","1. Uniform Cost search","\n","2. A* search(Misplaced Tile heuristic)","\n","3. A* search(Manhattan Distance heuristic)")
    choice=int(input("Enter your choice:"))
    print("\n")
    if choice==1:
        print("Uniform Cost Search")
        uniform_cost(start,goal)
    elif choice==2:
        print("A* search with Misplaced Tile heuristic")
        a_star_mtile(start,goal)
    else:
        print("A* search with Manhattan Distance heuristic")
        a_star_man(start,goal)

In [104]:
main()

Enter the difficulty of puzzle: 
 1. Trivial 
 2. Very Easy 
 3. Easy 
 4. Doable 
 5. Hard 
 6. Custom
Enter you choice: 5
Start state:
[[8 7 1]
 [6 0 2]
 [5 4 3]]
Goal state:
[[1 2 3]
 [4 5 6]
 [7 8 0]]
Enter the choice of algorithm you wish to run- 
 1. Uniform Cost search 
 2. A* search(Misplaced Tile heuristic) 
 3. A* search(Manhattan Distance heuristic)
Enter your choice:1


Uniform Cost Search
Success
Nodes expanded:  91121
Depth: 22
Solution found in: 242.7060  seconds
